In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import torch


In [15]:
# Load datasets
resumes = pd.read_csv('data/Resume Dataset/Resume/Resume_With_Skills.csv')
jobs = pd.read_csv('data/Linkedin Job Postings (2023-2024)/cleaned_JD_with_skills.csv')

# print column names
print(resumes.columns)
print(jobs.columns)




Index(['Resume_str', 'Category', 'skills', 'roles'], dtype='object')
Index(['roles', 'combined_skills_desc', 'skills', 'hard_skills'], dtype='object')


In [16]:
resumes["text"] = (
    resumes["Resume_str"] + " " +
    resumes["skills"].fillna("") + " " +
    resumes["roles"].fillna("")
)

jobs["text"] = (
    jobs["combined_skills_desc"] + " " +
    jobs["skills"].fillna("") + " " +
    jobs["roles"].fillna("")
)

In [17]:
# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

In [18]:
def get_embeddings_in_batches(text_list, batch_size=16):
    all_embeddings = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        all_embeddings.append(batch_embeddings)
    return np.vstack(all_embeddings)



In [12]:
resumes["text"] = resumes["text"].astype(str).fillna("")
jobs["text"] = jobs["text"].astype(str).fillna("")



In [19]:
# Compute embeddings
resume_embeddings = get_embeddings_in_batches(resumes["text"].tolist())
job_embeddings = get_embeddings_in_batches(jobs["text"].tolist())

# Rank Jobs

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

def rank_jobs(resume_text, job_embeddings, jobs, top_n=10):
    # Embed the resume
    resume_embedding = get_embeddings_in_batches([resume_text])
    
    # Compute cosine similarity
    similarities = cosine_similarity(resume_embedding, job_embeddings)
    
    # Get top N jobs
    top_indices = similarities.argsort()[0][-top_n:][::-1]
    top_scores = similarities[0, top_indices]
    
    return jobs.iloc[top_indices], top_scores

# Example resume of a nurse

resume_text = """

John Smith
Experienced Bank Manager
456 Financial Drive, Prosperity City, Wealth State, 67890
Phone: (987) 654-3210 | Email: john.smith@example.com | LinkedIn: linkedin.com/in/johnsmithbanking

Professional Summary
Results-driven Bank Manager with 10+ years of experience leading high-performing teams and driving financial growth in competitive banking environments. Skilled in risk management, customer relationship management (CRM), and operational efficiency. Proven ability to exceed sales targets while maintaining excellent customer satisfaction and compliance with regulatory standards.

Key Skills
Team Leadership & Staff Development
Financial Analysis & Risk Management
Branch Operations & Efficiency Optimization
Customer Relationship Management (CRM)
Compliance & Regulatory Adherence
Sales Strategy & Revenue Growth
Professional Experience
Branch Manager
Prosperity National Bank, Prosperity City, Wealth State
June 2015 – Present

Led a team of 20+ banking professionals to consistently exceed quarterly revenue and customer acquisition targets by 15%.
Managed daily operations of a $50M branch, ensuring compliance with state and federal banking regulations.
Developed and implemented sales strategies that increased loan and credit product sales by 20% year-over-year.
Fostered strong relationships with high-net-worth clients, securing over $5M in new deposits annually.
Assistant Branch Manager
Trust Financial Services, Wealth City, Wealth State
May 2012 – May 2015

Assisted in managing branch operations, focusing on staff training and customer service excellence.
Collaborated with the branch manager to develop local marketing campaigns, resulting in a 12% increase in foot traffic.
Conducted regular financial audits and reduced operational errors by 25%.
Resolved escalated customer complaints, achieving a 90% resolution satisfaction rate.
Education
Master of Business Administration (MBA) – Finance
Prestige Business School, Wealth City, Wealth State
Graduated: May 2012

Bachelor of Science in Economics
Prestige University, Wealth City, Wealth State
Graduated: May 2010

Certifications & Training
Certified Financial Manager (CFM) – National Institute of Banking
Leadership Development Program – American Bankers Association
Certified Anti-Money Laundering Specialist (CAMS)
Achievements
Awarded "Top Branch Manager of the Year" in 2020 and 2021 for highest customer retention and revenue growth.
Implemented a customer feedback program, increasing Net Promoter Score (NPS) by 30%.
Spearheaded a digital banking initiative, increasing mobile app usage by 40%.
 
"""

# Prepare the resume text for embedding
resume_text = resume_text.replace("\n", " ")


# Rank jobs
top_jobs, scores = rank_jobs(resume_text, job_embeddings, jobs)

# Display results
print("Top Job Matches:")
for job, score in zip(top_jobs["roles"], scores):
    print(f"{job}: {score:.2f}")


Top Job Matches:
Field Risk Specialist: 0.90
Field Risk Specialist: 0.90
Teller: 0.90
Data Project Manager: 0.90
Accounting Specialist: 0.90
Field Risk Specialist: 0.89
Security Specialist: 0.89
Personal Banker: 0.89
Relationship Banker: 0.89
Relationship Banker: 0.89
